# Arboles de Decisión


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from helpers import get_scores
from sklearn.grid_search import RandomizedSearchCV
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data

df, target = load_small_dev_data()
X = transformer.fit_transform(df)

print(X.shape)

(8099, 173)


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [2]:
from search import find_best_classifier
from sklearn.tree import DecisionTreeClassifier

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

search_options = {
    'cv': 10,
    'scoring': 'roc_auc',
    'n_jobs': 4,
    'n_iter': 20,
}

search = RandomizedSearchCV(DecisionTreeClassifier(), options, verbose=1, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=4)]: Done  76 tasks      | elapsed:    6.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   14.4s finished


Mejores parámetros: {'min_samples_split': 92, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 26, 'max_features': 0.50000000000000011}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
DecisionTreeClassifier,0.939307,0.942333,0.942531,0.945778,0.942333


# Haciendo PCA


Probemos ahora haciendo PCA sobre las variables, y luego aplicando el clasificador.

La reducción de dimensionalidad será una variable más del proceso de búsqueda de parámetros. Nos quedamos sólo con las variables de mayor componente principal. No sé si necesariamente es lo mejor esto...

In [3]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('pca', PCA()),
    ('tree', DecisionTreeClassifier()),
])
clf

Pipeline(steps=[('pca', PCA(copy=True, n_components=None, whiten=False)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])

Uso todas las opciones anteriores de árboles, pero tengo que agregarle un prefijo porque los meto en un pipeline (prefijo `tree__` para opciones del árbol, `pca__` para las de pca)

Voy a tomar como posibles dimensiones 20, 30, ..., n_dims - 10, n_dims

In [4]:
from helpers import add_prefix

new_options = add_prefix(options, 'tree__')

no_features = X.shape[1]

new_options.update({
    'pca__n_components' : range(20, no_features, 10)
})


search = RandomizedSearchCV(clf, new_options, verbose=1, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   17.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:  1.4min finished


Mejores parámetros: {'tree__criterion': 'entropy', 'tree__splitter': 'best', 'tree__min_samples_split': 62, 'pca__n_components': 50, 'tree__max_features': 0.90000000000000013, 'tree__max_depth': 10}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
Pipeline,0.932785,0.920667,0.91954,0.906667,0.920667
